In [2]:
# load packages
import os
import sys
import re
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_boston
tf.__version__

'1.12.0'

In [3]:
# load data
boston = load_boston()
features = np.array(boston.data)
labels = np.array(boston.target)
features.shape, labels.shape

((506, 13), (506,))

In [4]:
labels = labels.reshape((len(labels), 1))
labels.shape

(506, 1)

In [5]:
features[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [7]:
labels[0]

array([24.])

In [8]:
mean = np.mean(features, axis=0) # feature wise mean
std = np.std(features, axis=0) # feature wise standard deviation
# normalize feature now
features = (features - mean) / std
features.shape

(506, 13)

In [9]:
features[0]

array([-0.41978194,  0.28482986, -1.2879095 , -0.27259857, -0.14421743,
        0.41367189, -0.12001342,  0.1402136 , -0.98284286, -0.66660821,
       -1.45900038,  0.44105193, -1.0755623 ])

In [39]:
random_indices = np.random.rand(len(features)) < 0.80 # random indexes for splitting
# splti data
train_x = features[random_indices]
train_y = labels[random_indices]
val_x = features[~random_indices]
val_y = labels[~random_indices]

In [40]:
# set dimension
num_features = features.shape[1]
num_features

13

In [41]:
# create placeholders for input
x = tf.placeholder(dtype=tf.float32, shape=(None, num_features), name="feature_matrix") # num_samples x num_features
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="target_vector") # num_sampels X 1

In [42]:
# define weight for the model
w = tf.Variable(tf.zeros(dtype=tf.float32, shape=(1, num_features)))

In [43]:
# define bais
b = tf.Variable(tf.zeros(dtype=tf.float32, shape=(1, num_features)))

In [44]:
# linear model
linear_model = tf.add(tf.multiply(w, x), b)

In [45]:
loss = tf.reduce_mean(tf.square(linear_model - y)) # define loss function

In [46]:
# set optimizer for weight updates
trainer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [51]:
num_iterations = 10000

In [56]:
# run the mode
with tf.Session() as sess:
    print("[Training...]\n")
    sess.run(tf.global_variables_initializer()) # initialize variables
    # run the optimizer for 1000 iterations
    for i in range(num_iterations+1):
        sess.run(trainer, feed_dict={x:train_x, y:train_y})
        if i % 1000 == 0:
            print("Step", i, ":", end="\t")
            print("Train Loss: ", sess.run(loss, feed_dict={x:train_x, y:train_y}), end="\t")
            print("Validation Loss: ", sess.run(loss, feed_dict={x:val_x, y:val_y}))

[Training...]

Step 0 :	Train Loss:  615.5407	Validation Loss:  483.96448
Step 1000 :	Train Loss:  266.24237	Validation Loss:  181.4687
Step 2000 :	Train Loss:  122.7175	Validation Loss:  65.10624
Step 3000 :	Train Loss:  79.224884	Validation Loss:  39.27142
Step 4000 :	Train Loss:  72.752846	Validation Loss:  41.03009
Step 5000 :	Train Loss:  72.488625	Validation Loss:  42.67291
Step 6000 :	Train Loss:  72.48744	Validation Loss:  42.80791
Step 7000 :	Train Loss:  72.48744	Validation Loss:  42.809036
Step 8000 :	Train Loss:  72.48742	Validation Loss:  42.809196
Step 9000 :	Train Loss:  72.48741	Validation Loss:  42.8093
Step 10000 :	Train Loss:  72.487404	Validation Loss:  42.809357
